# Imports

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [2]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_normalized'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['age_scaled'] = df['age'] / df['age'].max()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')


variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [3]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [4]:
thresholds = [0, 1, 5, None]

ALPHA_0_1 = Beta('ALPHA_0_1', 0, None, None, 0)
ALPHA_1_5 = Beta('ALPHA_1_5', 0, None, None, 0)
ALPHA_5_INF = Beta('ALPHA_5_INF', 0, None, None, 0)
betas_piecewise = [ALPHA_0_1, 
                   ALPHA_1_5, 
                   ALPHA_5_INF]

piecewise_cost_driving = piecewise_formula(
    'cost_driving', thresholds, betas_piecewise
)

piecewise_cost_pt = piecewise_formula(
    'cost_transit', thresholds, betas_piecewise
)

In [5]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

G_TIME_WALK_AGE = Beta('G_TIME_WALK_AGE', 0, None, None, 0)
G_TIME_CYCLE_AGE = Beta('G_TIME_CYCLE_AGE', 0, None, None, 0) 
G_TIME_PT_AGE = Beta('G_TIME_PT_AGE', 0, None, None, 0)
G_TIME_DRIVE_AGE = Beta('G_TIME_DRIVE_AGE', 0, None, None, 0) 

# Utility functions with interaction terms
V_WALK = (B_TIME_WALK + G_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + G_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + piecewise_cost_pt + (B_TIME_PT + G_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE +  piecewise_cost_driving + (B_TIME_DRIVE + G_TIME_DRIVE_AGE * age_scaled) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

logprob_m4 = loglogit(V, availability, travel_mode)

model_4 = bio.BIOGEME(database, logprob_m4)
model_4.modelName = 'model_4'

results_m4 = model_4.estimate()
model_4_loglike = results_m4.data.logLike
model_4_numParam = results_m4.get_estimated_parameters().shape[0]
model_4_table = results_m4.get_estimated_parameters()

model_4_table

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ALPHA_0_1,-0.694669,0.084508,-8.220165,2.220446e-16
ALPHA_1_5,-0.040249,0.052875,-0.761206,4.465342e-01
ALPHA_5_INF,-0.203265,0.038237,-5.315889,1.061379e-07
ASC_CYCLE,-4.720183,0.208686,-22.618594,0.000000e+00
ASC_DRIVE,-2.102498,0.150318,-13.986984,0.000000e+00
ASC_PT,-2.431676,0.153179,-15.874779,0.000000e+00
B_TIME_CYCLE,-5.030458,0.901862,-5.577855,2.435020e-08
B_TIME_DRIVE,-5.882143,0.882375,-6.666265,2.623946e-11
B_TIME_PT,-2.621283,0.487938,-5.372167,7.779602e-08
B_TIME_WALK,-7.762626,0.514986,-15.073481,0.000000e+00


In [6]:
data_filtered = df

populations = {
    'female_45_less': 2841376,
    'female_45_or_more': 1519948,
    'male_45_less': 2929408,
    'male_45_or_more': 1379198,
}

total_pop = sum(populations.values())

filters = {
    'male_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 0),
    'male_45_less': (data_filtered.age < 45) & (data_filtered.female == 0),
    'female_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 1),
    'female_45_less': (data_filtered.age < 45) & (data_filtered.female == 1),
}

sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(sample_segments)

total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

weights = {
    segment_name: populations[segment_name] * total_sample / (segment_size * total_pop)
    for segment_name, segment_size in sample_segments.items()
}
print(weights)

{'male_45_or_more': np.int64(896), 'male_45_less': np.int64(1442), 'female_45_or_more': np.int64(984), 'female_45_less': np.int64(1678)}
Sample size: 5000
{'male_45_or_more': np.float64(0.8877139043468962), 'male_45_less': np.float64(1.1715720875375348), 'female_45_or_more': np.float64(0.890816074423909), 'female_45_less': np.float64(0.9765425353056789)}


In [7]:
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp

from biogeme import models

for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]


prob_walk = models.logit(V, None, 1)
prob_cycle = models.logit(V, None, 2)
prob_pt = models.logit(V, None, 3)
prob_car = models.logit(V, None, 4)


weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
}

# data_filtered["cost_driving_fuel"] += 1.5
database = db.Database('london', df)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)


market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.2f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.2f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.2f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.2f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)
    
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.887714,0.325063,0.651198,1.718331e-03,0.022020
1,0.976543,0.126739,0.685906,1.351130e-01,0.052242
2,0.890816,0.198557,0.783280,1.151712e-06,0.018163
3,0.887714,0.379195,0.577307,2.965678e-03,0.040532
4,0.890816,0.198323,0.335469,4.400427e-01,0.026166
...,...,...,...,...,...
4995,0.887714,0.298430,0.659737,6.320184e-03,0.035513
4996,1.171572,0.385400,0.521252,2.371531e-02,0.069632
4997,0.976543,0.209348,0.391852,3.474200e-01,0.051380
4998,0.976543,0.263058,0.719631,4.966602e-05,0.017261


Market share for pt: 35.73%
Market share for car: 43.53%
Market share for walk: 17.45%
Market share for cycling: 3.29%


100%|██████████| 100/100 [01:23<00:00,  1.19it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.887714,0.292797,0.615553,5.983084e-04,0.017215
1,0.976543,0.109482,0.664852,1.134672e-01,0.043953
2,0.890816,0.171154,0.759138,1.738581e-07,0.012857
3,0.887714,0.351537,0.548018,1.264384e-03,0.034003
4,0.890816,0.181045,0.316234,4.118656e-01,0.020474
...,...,...,...,...,...
4995,0.887714,0.280232,0.646304,3.305479e-03,0.030605
4996,1.171572,0.362278,0.499487,1.539720e-02,0.060896
4997,0.976543,0.194593,0.376550,3.258525e-01,0.043437
4998,0.976543,0.246740,0.706248,1.577523e-05,0.013536


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle
0,0.887714,0.359916,0.680874,4.084403e-03,0.027491
1,0.976543,0.144620,0.708537,1.489033e-01,0.059414
2,0.890816,0.224624,0.812230,3.731239e-06,0.023862
3,0.887714,0.410517,0.602720,5.498701e-03,0.047797
4,0.890816,0.217817,0.350767,4.632115e-01,0.031142
...,...,...,...,...,...
4995,0.887714,0.312987,0.678898,9.574553e-03,0.040040
4996,1.171572,0.406800,0.541408,3.089561e-02,0.078703
4997,0.976543,0.222408,0.405953,3.671714e-01,0.059500
4998,0.976543,0.277542,0.736331,9.840776e-05,0.020760


In [8]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.2f}% "
      f"CI: [{100 * left_market_share_pt:.2f}%-{100 * right_market_share_pt:.2f}%]")

print(f"Market share for car: {100 * market_share_car:.2f}% "
      f"CI: [{100 * left_market_share_car:.2f}%-{100 * right_market_share_car:.2f}%]")

print(f"Market share for walk: {100 * market_share_walk:.2f}% "
      f"CI: [{100 * left_market_share_walk:.2f}%-{100 * right_market_share_walk:.2f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.2f}% "
      f"CI: [{100 * left_market_share_cycle:.2f}%-{100 * right_market_share_cycle:.2f}%]")

Market share for pt: 35.73% CI: [33.39%-38.14%]
Market share for car: 43.53% CI: [41.06%-46.07%]
Market share for walk: 17.45% CI: [16.13%-18.76%]
Market share for cycling: 3.29% CI: [2.67%-3.90%]


In [9]:
labels = {1: 'walk', 2: 'cycling', 3: 'pt', 4: 'car'}


# Map the travel_mode column to the labels
data_filtered['mode_label'] = data_filtered['travel_mode'].map(labels)

# Calculate market shares
market_shares = (
    data_filtered['mode_label']
    .value_counts(normalize=True)  # Get proportions
    .sort_index()  # Ensure consistent order
    * 100  # Convert to percentage
)

# Print market shares
for mode, share in market_shares.items():
    print(f"Market share for {mode}: {share:.2f}%")



Market share for car: 43.98%
Market share for cycling: 3.28%
Market share for pt: 35.32%
Market share for walk: 17.42%


In [10]:
# Example mapping
labels = {1: 'walk', 2: 'cycling', 3: 'pt', 4: 'car'}

# Map the travel_mode column to the labels
data_filtered['mode_label'] = data_filtered['travel_mode'].map(labels)

# Calculate total weight across all individuals
total_weight = data_filtered['weight'].sum()

# Calculate weighted market shares
market_shares = (
    data_filtered.groupby('mode_label')['weight']  # Group by travel mode
    .sum()  # Sum weights for each mode
    / total_weight  # Divide by total weight to get the proportion
    * 100  # Convert to percentage
)

# Print market shares
for mode, share in market_shares.items():
    print(f"Market share for {mode}: {share:.2f}%")


Market share for car: 43.53%
Market share for cycling: 3.45%
Market share for pt: 35.49%
Market share for walk: 17.53%
